In [ ]:
import requests
import pandas as pd
import python_quant_super_value as pq
import time
import bs4
import numpy as np
import urllib.parse
import pandas as pd
import matplotlib.pyplot as plt
import numpy

In [ ]:
import imp
import python_quant_super_value
imp.reload(python_quant_super_value)

## 전 종목 코드 구해 오기 (Python)

[참조] 파이썬으로 전 종목 코드 구하는 방법
https://minjejeon.github.io/learningstock/2017/09/07/download-krx-ticker-symbols-at-once.html

In [ ]:
kospi = pq.download_stock_codes('kospi')
kosdaq = pq.download_stock_codes('kosdaq')
#konex = pq.download_stock_codes('konex')
code_data = pd.concat([kospi, kosdaq])[['종목코드','회사명']]
#code_data = pd.concat([kospi, kosdaq, konex])[['종목코드','회사명']]
code_data.columns = ['종목코드','기업명']
code_data['종목코드'] = 'A' + code_data['종목코드']
code_data = code_data.set_index(code_data.columns[0])

# code_data.to_csv('code_data.csv', encoding='cp949', index = False)
# code_data2 = pd.read_csv('code_data.csv', encoding='cp949')
# code_data2

In [ ]:
code_data.shape

In [ ]:
code_data.head(10)

## code test

In [ ]:
pq.make_cap_dataframe("A005720")

In [ ]:
pq.make_fs_dataframe('A294140') 

In [ ]:
pq.make_fhd_dataframe('A353200')

## fnguide 데이터 수집하기

In [ ]:
# [코드 3.27/29/31] 모든 종목에 대해서 재무제표, 재무비율, 투자지표 데이터 가져오기 (CH3. 데이터 수집하기.ipynb)

for num, code in enumerate(code_data.index):
    try:
        print(num, code)
        time.sleep(1)        
        try:
            cap = pq.make_cap_dataframe(code)
            fs_df = pq.make_fs_dataframe(code)
            fhd_df = pq.make_fhd_dataframe(code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            cap = pq.make_cap_dataframe(code)
            fs_df = pq.make_fs_dataframe(code)
            fhd_df = pq.make_fhd_dataframe(code)
        except ValueError:
            continue
        except KeyError:
            continue
        except UnboundLocalError:
            continue

        value = pd.merge(cap, fs_df, how='outer', right_index=True, left_index=True)
        value = pd.merge(value, fhd_df, how='outer', right_index=True, left_index=True)
        value['1/PSR'] = float(value['매출액'][0]) / float(value['시가총액'][0])
        value['1/PER'] = float(value['순이익'][0]) / float(value['시가총액'][0])
        value['1/PBR'] = float(value['자본'][0]) / float(value['시가총액'][0])
        value['1/PCR'] = float(value['영업현금흐름'][0]) / float(value['시가총액'][0])
        value = pd.merge(code_data.loc[[code]], value, how='outer', right_index=True, left_index=True)
        if num == 0 :
            total_value = value
        else:
            total_value = pd.concat([total_value, value])

    except ValueError:
        continue
    except KeyError:
        continue
    except UnboundLocalError:
        continue


In [ ]:
total_value

In [ ]:
total_value['적정가'] = round((total_value['BPS'].astype('float')*(1+total_value['ROE3년']*0.01-0.05*((100+total_value['부채비율3년'])/100)**0.5)**10).astype(float), -1)

In [ ]:
'data\total_value_{}.xlsx'.format(day.strftime("%y%m%d"))

In [ ]:
day.strftime("%y%m%d")

In [ ]:
total_value.to_excel(r'data\total_value_{}.xlsx'.format(day.strftime("%y%m%d")))
total_value.shape[0]

In [ ]:
total_value

## super value 전략 구현하기

In [ ]:
fs_path = r'data\total_value_{}.xlsx'.format(day.strftime("%y%m%d"))
value_df = pd.read_excel(fs_path)
sorted_cap_value = pq.get_small_cap(value_df, ratio=[0.8,1.0])
value_list = ['1/PSR','1/PER','1/PBR','1/PCR']
value_combo = pq.value_combo(sorted_cap_value, value_list, 55)
value_combo


In [ ]:
value_combo.to_excel(r'data\value_combo_{}.xlsx'.format(day.strftime("%y%m%d")))